In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

os.environ["HF_HOME"] = "../models/"
os.environ["TORCH_HOME"] = "../models/"
os.environ["TRANSFORMERS_OFFLINE"] = "1"

In [3]:
from til_23_asr import VoiceExtractor
from IPython.display import Audio
import librosa
import torch
from pathlib import Path

In [4]:
# load original and noisy speech files
files = list(Path('../data/audio/train').glob('*.wav')) + list(Path('../data/audio/val').glob('*.wav'))
# files = Path('/mnt/e/Downloads/simulator/orgteam1_vs_orgteam2/valid_checkpoint_8/Digits').glob('*.wav')
wavs = {}
for file in sorted(files):
    noisy, sr = librosa.load(file, sr=None)
    wavs[file.stem] = (noisy, sr)

In [5]:
extractor = VoiceExtractor(
    model_dir="../models/DeepFilterNet3",
    sr=sr,
    # spectral_gate_std_thres=1.5,
    # spectral_noise_remove=1.0,
    # spectral_n_fft=512,
    # spectral_freq_mask_hz=None,
    # spectral_time_mask_ms=None,
    df_post_filter=False,
).to("cuda")

fatal: not a git repository (or any of the parent directories): .git


In [6]:
def test(name, wav, sr):
    print(name)
    display(Audio(noisy, rate=sr))
    clean, new_sr, noise = extractor.forward(
        torch.tensor(wav, device="cuda"),
        sr,
        skip_vol_norm=False,
        skip_df=False,
        # Spectral is now useless; Noise sample is fine most of the time except PALMTREE.
        skip_spectral=True,
        use_ori=False,
        # Should be 0 for all except PALMTREE.
        noise_removal_limit_db=0,
        return_noise=True,
    )
    display(Audio(clean.numpy(force=True), rate=new_sr))
    display(Audio(noise.numpy(force=True), rate=new_sr))
    return wav

In [ ]:
for name, (noisy, sr) in wavs.items():
    if "PALMTREE" not in name:
        continue
    test(name, noisy, sr)

In [ ]:
demucked, d_sr = extractor._demucs(torch.tensor(noisy), sr)
display(Audio(demucked, rate=d_sr))

In [ ]:
display(Audio(extractor._spectral(demucked, d_sr)[0], rate=sr))